In [ ]:
import pandas as pd
import os
from glob import glob
import numpy as np
import json
from itertools import izip

In [2]:
def get_env_var(handle):
    ''' Get an environment variable given the handle for the bash variable'''
    tmp = os.getenv(handle)
    if not tmp:
        raise LookupError("Environment variable: {} not set.".format(handle))
    return tmp.strip("'")

def format_md_img(link_name, rel_plot_loc):
    img_formatter="[[{}/{}]]".format(link_name, rel_plot_loc)
    return img_formatter

def print_table(list_of_headers):
    fmter =  "| {} {}".format("{}", " {} ")
    for header in list_of_headers:
        fmter = fmter.format("{} {}".format(header, "| {}"), "{}")
    fmter = fmter.format(" ", " ")
    return fmter

class  Experiment(dict):
    def __init__(self, name):
        self.name = name
        self.folds = []
        self.converged_epochs = []
        self.trained_paths = []
      
    def __repr__(self):
        return str(vars(self))
    
    def __str__(self):
        return json.dumps(vars(self), indent=2)
        
    def set_convereged_epoch(self, epoch, train_path, fold=None):        
        self.folds.append(fold)
        self.converged_epochs.append(epoch)
        self.trained_paths.append(train_path)
        

In [43]:
home=get_env_var("HOME")
base="{}/labgits/lab-notebook-caceres.wiki/images/20180815_Paper_Retrains/".format(home)

In [81]:
home_save_dir = get_env_var("HOME_SAVE_BASE")
srv_save_dir = get_env_var("DATA_SAVE_BASE")
proj_dir = get_env_var("NMEW_PROJ_BASE")

sneg_pnrs=[1.0, 1.2222, 0.8182, 1.5, 19.0, 9.0, 4.0, 2.3333, 0.6666, 0.4286, 0.25, 0.1111, 0.0753]
sneg_pnrs = sorted(sneg_pnrs)
fold_tmplt = "fold_[0-9]*/pnr_*/*.png"

# github formatting info:
github_wiki_link="https://github.com/keiserlab/lab-notebook-caceres/wiki/images"
expt_name = "20180815_Paper_Retrains"
expt_sub_name="STD_SMA_RATIOS"
github_wiki_expt = "{}/{}".format(github_wiki_link, expt_name, expt_sub_name)

neg_rm_dir = "{}/lr_nesterov_1024_2048_3072/".format(base)

metrics_for_convergance = ['matthews-corrcoef_binary-5.0_test', 'matthews-corrcoef_binary-6.0_test', 'r2_test']
experiments = []

parent_dir = os.path.join(neg_rm_dir, expt_sub_name)

### copied images of interest into parent dir

```
!rsync -azrRv $HOME_SAVE_BASE/output/20180815_Paper_Retrains/trained_nets/./lr_nesterov_1024_2048_3072/STD_SMA_RATIOS/fold_*/pnr_*/*.png $base
```

In [82]:
png_files = glob(os.path.join(parent_dir, fold_tmplt))

In [83]:
experiments = []
converged_epochs = ("{}/output/20180815_Paper_Retrains/predictions/{}/experiments.json".format(home_save_dir, expt_sub_name))
with open(converged_epochs, "r") as fp:
    expts = json.load(fp)

expt_list = []
for e in expts:
    tmp = Experiment(e["name"])
    for epoch, path, fold in izip(e["converged_epochs"], e["trained_paths"], e["folds"]):
        tmp.set_convereged_epoch(epoch, path, fold)
    experiments.append(tmp)
    del(tmp)    
    
expt_dict = {float(i.name.split("_")[-1]): dict(zip([int(j.split("_")[1]) for j in i.folds], i.converged_epochs)) for i in experiments}

In [84]:
expt_dict

{0.0753: {0: 424, 1: 468, 2: 499, 3: 229, 4: 280},
 0.1111: {0: 480, 1: 292, 2: 209, 3: 429, 4: 120},
 0.25: {0: 358, 1: 262, 2: 381, 3: 161, 4: 321},
 0.4286: {0: 228, 1: 245, 2: 354, 3: 458, 4: 212},
 0.6666: {0: 213, 1: 261, 2: 108, 3: 96, 4: 98},
 0.8182: {0: 400, 1: 224, 2: 299, 3: 260, 4: 139},
 1.0: {0: 200, 1: 316, 2: 100, 3: 294, 4: 344},
 1.2222: {0: 236, 1: 81, 2: 160, 3: 242, 4: 173},
 1.5: {0: 97, 1: 206, 2: 172, 3: 373, 4: 72},
 2.3333: {0: 132, 1: 0, 2: 0, 3: 199, 4: 0},
 4.0: {0: 356, 1: 300, 2: 211, 3: 362, 4: 339},
 9.0: {0: 263, 1: 81, 2: 354, 3: 0, 4: 296},
 19.0: {0: 57, 1: 459, 2: 194, 3: 100, 4: 206}}

In [85]:
img_order = ['train_loss','test_loss', 'test_no_gt_loss', 'test_sneg_loss']
for ratio in sneg_pnrs:
    png_subset = [f for f in png_files if str(ratio) in f]
    headers = ["PNR_{}".format(ratio)]
    headers.extend(img_order)
    print(print_table(headers))
    print(print_table([":---"]*len(headers)))
    for f in np.arange(0, 5, 1):
        name_fmter = "fold_{} best epoch: {}".format(str(f), expt_dict[ratio][f])
        fold_fmter = "fold_{}".format(str(f))
        fold_pngs = [i for i in png_subset if fold_fmter in i]
        fold_pngs = sorted(fold_pngs, key=lambda x: img_order.index(x.split("/")[-1].split(".")[0]))
        md_print_fmt = [name_fmter] + [format_md_img(github_wiki_expt, i.split("//")[-1]) for i in fold_pngs]
        print(print_table(md_print_fmt))
    print("\n\n")

| PNR_0.0753 | train_loss | test_loss | test_no_gt_loss | test_sneg_loss |      
| :--- | :--- | :--- | :--- | :--- |      
| fold_0 best epoch: 424 | [[https://github.com/keiserlab/lab-notebook-caceres/wiki/images/20180815_Paper_Retrains/lr_nesterov_1024_2048_3072/STD_SMA_RATIOS/fold_0/pnr_0.0753/train_loss.png]] | [[https://github.com/keiserlab/lab-notebook-caceres/wiki/images/20180815_Paper_Retrains/lr_nesterov_1024_2048_3072/STD_SMA_RATIOS/fold_0/pnr_0.0753/test_loss.png]] | [[https://github.com/keiserlab/lab-notebook-caceres/wiki/images/20180815_Paper_Retrains/lr_nesterov_1024_2048_3072/STD_SMA_RATIOS/fold_0/pnr_0.0753/test_no_gt_loss.png]] | [[https://github.com/keiserlab/lab-notebook-caceres/wiki/images/20180815_Paper_Retrains/lr_nesterov_1024_2048_3072/STD_SMA_RATIOS/fold_0/pnr_0.0753/test_sneg_loss.png]] |      
| fold_1 best epoch: 468 | [[https://github.com/keiserlab/lab-notebook-caceres/wiki/images/20180815_Paper_Retrains/lr_nesterov_1024_2048_3072/STD_SMA_RATIOS/fold_1/pnr